# Using Python and SQL

An advantage of using a popular language is the high probability that programmers before us have found solutions to our problems already. As such, there exists a Python module which allows us to interface with a PostgreSQL database, called Psycopg2.

Using this module requires a quite advanced understanding of Python and SQL, so we'll start off with a very simple query and work through it step by step.

Let's suppose we want to implement the following SQL query:

<span style="color:orange">**SELECT**</span> 2 + 3;

To run this SQL query in Python, we use psycopg2 as follows:

In [ ]:
import psycopg2

# Establish the connection
conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()

# Execute an SQL query and receive the output
cursor.execute('SELECT 2 + 3;')
records = cursor.fetchall()

print(records)

[(5,)]

N.B. That grok is the online tool in the AstroMOOC.

And that's our expected result. Now let's look at each line in detail.

We will go through the example on the last slide step by step.

**1. Establish a connection to the database**

In [ ]:
conn = psycopg2.connect(dbname='db', user='grok')

This command initialises a new database session and returns a connection object. We have to specify the name of the database and the name of the user. Note that the dbname is the name of the database, not a table in the database.

Throughout this module we're calling our database 'db'. On your local machine, you would use your user account name for the user. Here we're going to use 'grok'.

**2. Create a cursor object**

In [ ]:
cursor = conn.cursor()

The cursor is the object that interfaces with the database. We can execute SQL queries and receive their output through this object. We can call the object's functions by using the dot (.) notation just like we do for modules. The two functions that we will use most often are execute and fetchall.

**3. Run a SQL query**

In [ ]:
cursor.execute('SELECT 2 + 3;')

To run a SQL query, we call the execute function, which is a function of the cursor object. This function takes the SQL query in form of a string as its argument.

**4. Receive the query return**

In [ ]:
records = cursor.fetchall()

The fetchall function returns the output of the last query. When taking SQL data into Python, the data types are converted to the closest match in Python data types. We'll have a closer look at this later.

To get started with basic Psycopg2 usage, write a function called select_all which queries either our Star or Planet table in PostgreSQL and returns all the rows using the following query:

<span style="color:orange">**SELECT**</span> * <span style="color:orange">**FROM**</span> Star;

Your function should take the name of the table as a string argument, so you can call it like to access the Star table:

In [ ]:
>>> select_all('Star')
[(2713049, 5996, 0.956), (3114167, 5666, 0.677), (3115833, 5995, 0.847), ...]

Or like this for the Planet table:

In [ ]:
>>> select_all('Planet')
[(10666592, 'K00002.01', 'Kepler-2b', 'CONFIRMED', 2.204735365, 16.39, 2025), ...]

It should return the result of **cursor.fetchall()** directly.

In [ ]:
# Establish the connection
conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()

# Define select_all function
def select_all(Table_name):
    query = 'SELECT * FROM ' + Table_name + ';'
    cursor.execute(query)
    return cursor.fetchall()

As you've seen in the last problem, the data from SQL queries in Psycopg2 is returned in form of Python lists. In the last problem, you requested the full Star and Planet table, which returned a list of n tuples of length m, where m is the number of columns and n is the number of rows in these tables.

A list of tuples cannot be used in the same way as e.g. a 2D Numpy array. For example, the following method of indexing to access the first element will not work:

In [ ]:
a = [(1, 2, 3), (4, 5, 6)]
print(a[0, 0])

Instead, we have to use the [] operator twice: first to access the first list element, i.e. the first tuple, and then to access the first element in that tuple:

In [ ]:
a = [(1, 2, 3), (4, 5, 6)]
print(a[0][0])

Using this indexing method, we can then access every individual data element. This allows us to, e.g. extract entire columns of the data by looping over the rows. The following code snippet shows an example which extracts the t_eff column from the full Star table and appends it to a new list:

In [ ]:
import psycopg2

conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()
cursor.execute('SELECT * FROM Star')
records = cursor.fetchall()

t_eff = []
for row in records:
  t_eff.append(row[1])

print(t_eff)

Now we've seen how to work with query results, we can have a closer look at the data itself. In the previous activity, we learned about different data types in SQL when we were setting up tables.

How do these SQL data types get converted into Python types?

Let's have a look at the Planet table's data types. We can use a query which selects all columns but only a single row:

In [ ]:
+-----------+-----------+-------------+-----------+-------------+--------+------+
| kepler_id | koi_name  | kepler_name |  status   |   period    | radius | t_eq |
+-----------+-----------+-------------+-----------+-------------+--------+------+
|  10666592 | K00002.01 | Kepler-2b   | CONFIRMED | 2.204735365 |  16.39 | 2025 |
+-----------+-----------+-------------+-----------+-------------+--------+------+
(1 row)

In Python, this query will return a list containing a single tuple. We can loop over the entries of this tuple and call the type function to determine the data types:

In [ ]:
import psycopg2

conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()

cursor.execute('SELECT * FROM Planet LIMIT 1;')

records = cursor.fetchall()

for col in records[0]:
    print(type(col))

In [ ]:
<type 'int'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'float'>
<type 'float'>
<type 'int'>

The type conversion of these types is straight-forward: SQL's SMALLINT and INTEGER get converted to Python integers, CHAR and VARCHAR to Python strings, and FLOAT to Python floats.

Check out the Psycopg2 documentation when you want to learn about type conversion in more detail.

Once we have the numerical data from the database in Python, we can write them into NumPy arrays.

Since we're often dealing with data of different types in databases, it is important to remember that while Python lists and tuples can hold data of different types, NumPy arrays cannot.

To convert a Python list into a simple NumPy array, we must ensure that the list only contains data of one type. Other than that, SQL results can easily be loaded into NumPy arrays:

In [ ]:
import psycopg2
import numpy as np

conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()

cursor.execute('SELECT radius FROM Star;')

records = cursor.fetchall()
array = np.array(records)

print(array.shape)
print(array.mean())
print(array.std())

In [ ]:
(66, 1)
0.886863636364
0.237456527847

Write a function called column_stats which calculates the mean and median of a selected column in either Star or Planet table. For this, let your function take two string arguments:

the name of the table;
the name of the column.
and have it return the mean and median (in this order) of the selected column.

When you call your function on, for example, the t_eff column of the Star table, the function call and return should look like this:

In [ ]:
>>> column_stats('Star', 't_eff')
(5490.681818181818, 5634.0)

You can compare your calculation with the pure SQL query:

<span style="color:orange">**SELECT**</span> AVG(t_eff) <span style="color:orange">**FROM**</span> Star;

In [ ]:
+-----------------------+
|          avg          |
+-----------------------+
| 5490.6818181818181818 |
+-----------------------+
(1 row)

In [ ]:
# My function to give column mean and median from one variable in one table (works fine)
def column_stats(Table, variable):
    query = 'SELECT ' + variable + ' FROM ' + Table + ';'
    cursor.execute(query)
    
    records = cursor.fetchall()
    array = np.array(records)
    
    return np.mean(array), np.median(array)

print(column_stats('Star', 't_eff'))

In this course you've learned two different approaches to dealing with data. Which you choose for a particular project depends on a variety of factors including the questions you're posing of the data or whether you're using a public database or catalogue.

We have seen that SQL is convenient to use for a lot of things – but exactly how convenient is it? Can we do the same thing in Python?

Let's go through a few problems in which we implement typical SQL queries from the previous activities in Python. We will start of with a simple query and add a new element in each problem.

Your first task is to replicate the following SQL query:

In [ ]:
SELECT kepler_id, radius
FROM Star
WHERE radius > 1.0;

The data is stored in stars.csv, with the kepler_id in the first column and the radius in the last.

Write a function called query which takes the CSV filename as an argument and returns the data in a 2-column NumPy array. For example, this small CSV file:

In [ ]:
stars.csv

10666592,6350,1.991
10682541,5339,0.847
10797460,5850,1.04

your **query** function should work as follows:

In [ ]:
>>> query('stars.csv')
array([[  1.06665920e+07   1.99100000e+00]
       [  1.07974600e+07   1.04000000e+00]])

The numerical data gets automatically converted to floats in this procedure, don't worry if it doesn't look like the SQL output.

**Hint:** You can use NumPy's loadtxt function with the optional usecols argument to read in only those columns you're interested in.

In [ ]:
import numpy as np

# Write your query function here. 
# I used the given solution here as better than my one in its use of numpy
def query(file_name):
    data = np.loadtxt(file_name, delimiter=',', usecols=(0, 2)))
    return data[data[:, 1]>1, :]

Let's add another element to our query. Sort the resulting table in ascending order to match the result you would get with:

In [ ]:
SELECT kepler_id, radius
FROM Star
WHERE radius > 1.0
ORDER BY radius ASC;

You can use your results from the last problem and then build up on that. Again, the function should be named query and it should take the filename as argument.

**Hint:**
You can use NumPy's argsort function to solve this problem. Take a look at how it works:

In [9]:
import numpy as np
  
a = np.array([3, 1, 2, 0])
b = np.argsort(a)
print(b)
print(a[b])

[3 1 2 0]
[0 1 2 3]


^ Nice way to index!

My function:

In [ ]:
import numpy as np

# Write your query function here
def query(file_name):
    data = np.loadtxt(file_name, delimiter=',', usecols=(0, 2))
    data = data[data[:, 1]>1, :]
    return data[np.argsort(data[:, 1])]

#print(query('stars.csv'))

Let's add yet another element to our query. Join the Star table with the Planet table and calculate the size ratio, i.e. planet radius / star radius for each star-planet pair. Your query function should produce the same result as the SQL query:

In [ ]:
SELECT p.radius/s.radius AS radius_ratio
FROM Planet AS p
INNER JOIN star AS s USING (kepler_id)
WHERE s.radius > 1.0
ORDER BY p.radius/s.radius ASC;

You can use your results from the last problem and then build up on that. The function must be named query, but this time it should take two filenames as arguments, for the stars and planets.

In planets.csv, the first column is the kepler_id and the second last column is the radius.

Your function should be a column vector of ratios, like this:

In [ ]:
>>> query('stars.csv', 'planets.csv')
array([[  0.48798799],
       [  0.8260447 ],
       [  0.96209913],
       [  1.1556384 ],
       [  1.30403969],
       ...

**Hint:** You may need to use a nested loop to compare each Planet's kepler_id against each Star's kepler_id. Once you've found a match and the star's radius is larger than one, you can append the ratio to the results list or array.

In [ ]:
# My working function here:
# See akuederle comment in: http://akuederle.com/create-numpy-array-with-for-loop:
# "The easiest way is to create a simple python list of lists and convert it to an numpy array afterwards.""

import numpy as np

def query(fname_1, fname_2):
  stars = np.loadtxt(fname_1, delimiter=',', usecols=(0, 2))
  planets = np.loadtxt(fname_2, delimiter=',', usecols=(0, 5))
  
  # get stars with radius > 1 and sort
  stars = stars[stars[:,1]>1, :]                
  stars_sorted = stars[np.argsort(stars[:, 1]), :] 
  planets_sorted = planets[np.argsort(planets[:,1]),:]
                                            
 
  # iterate by lists as easier, then make as an array
  # interestingly, could iterate over arrays!
  output = []
  for row_s in stars_sorted:
      for row_p in planets_sorted:
          if row_s[0] == row_p[0]:
             output.append([row_p[1]/row_s[1]])
  
  output.sort()
  z = np.asarray(output)            
  return(z)